In [1]:
import os
import random
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest

In [2]:
#def set_seed(seed: int):
#    random.seed(seed) # Python
#    np.random.seed(seed)  # Numpy, é o gerador utilizado pelo sklearn
#    os.environ["PYTHONHASHSEED"] = str(seed)  # sistema operativo
#
#set_seed(25)

In [3]:
# tratamento da seed - pytorch
def set_seed(seed: int, deterministic_torch: bool = True):
    random.seed(seed) # Python
    np.random.seed(seed)  # Numpy, é o gerador utilizado pelo sklearn
    os.environ["PYTHONHASHSEED"] = str(seed)  # sistema operativo
    if torch is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  #multi-GPU
        if deterministic_torch:
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

set_seed(25)

In [4]:
# Load dataset
radi = pd.read_csv("../sbsppdaa24/train_radiomics_hipocamp.csv")
radi_test = pd.read_csv("../sbsppdaa24/test_radiomics_hipocamp.csv")

In [5]:
# Process the train dataset
radi.drop(columns=["Mask", "ID", "Image"], inplace=True)

# Drop non-numeric columns except for 'Transition'
columns_to_drop = [col for col in radi.columns if radi[col].dtype == 'object' and col != 'Transition']
radi.drop(columns=columns_to_drop, inplace=True)
print(f"Dropped {len(columns_to_drop)} non-numeric columns.")

# Apply MinMax scaling to float columns
float_cols = radi.select_dtypes(include=['float', 'int']).columns
scaler = MinMaxScaler()
radi[float_cols] = scaler.fit_transform(radi[float_cols])

# Feature selection using ANOVA 
X = radi.drop(columns=["Transition"])
y = radi["Transition"]
selector = SelectKBest(score_func=f_classif, k=100)  # Selecionar as 100 melhores features
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print(f"Selected features: {selected_features.tolist()}")

# Atualizar o dataset com as features selecionadas
radi = pd.concat([pd.DataFrame(X_new, columns=selected_features), y.reset_index(drop=True)], axis=1)

# Save the full train dataset
radi.to_csv("train_full_prep7.csv", index=False)

# Split into 80% train and 20% test
radi_train, radi_val = train_test_split(radi, test_size=0.2, random_state=25,stratify=radi['Transition'])
radi_train.to_csv("train_split_prep7.csv", index=False)
radi_val.to_csv("test_split_prep7.csv", index=False)

# Process the test dataset
radi_test.drop(columns=["Mask", "ID", "Image"], inplace=True)
radi_test.drop(columns=[col for col in radi_test.columns if radi_test[col].dtype == 'object' and col != 'Transition'], inplace=True)

# Apply MinMax scaling to the test dataset
radi_test[float_cols] = scaler.transform(radi_test[float_cols])  # Use the same scaler fit on the train dataset
radi_test =radi_test[selected_features]
radi_test.to_csv("test_processed_prep7.csv", index=False)

print("Datasets processed and saved:")
print("- train_full.csv")
print("- train_split.csv")
print("- test_split.csv")
print("- test_processed.csv")


Dropped 16 non-numeric columns.


/Users/nunorodrigues/miniconda3/envs/myenv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [   0    2    5 1790 1793 1797 1798 1803 1805 1806 1807 1808 1809 1810
 1811 1812 1813 1814 1815 1816 1817 1818 1819 1820 1821 1822 1823 1824
 1825 1826 1827 1828 1834 1835 1839 1844 1845 1846 1850 1859 1860 1861
 1862 1866 1867 1868 1872 1875 1876 1877 1878 1879 1883 1886 1890 1891
 1896 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910
 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1927 1928 1932
 1937 1938 1939 1943 1952 1953 1954 1955 1959 1960 1961 1965 1968 1969
 1970 1971 1972 1976 1983 1989 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2020 2021 2025 2030 2031 2032 2036 2045 2046 2047 2048 2052
 2053 2054 2058 2061 2062 2063 2064 2065] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/U

Selected features: ['original_shape_SurfaceVolumeRatio', 'original_firstorder_Energy', 'original_firstorder_TotalEnergy', 'original_gldm_SmallDependenceEmphasis', 'original_glszm_LargeAreaEmphasis', 'original_glszm_ZoneVariance', 'wavelet-LHL_ngtdm_Coarseness', 'wavelet-LHH_gldm_DependenceEntropy', 'wavelet-LHH_gldm_DependenceNonUniformity', 'wavelet-LHH_gldm_DependenceNonUniformityNormalized', 'wavelet-LHH_gldm_DependenceVariance', 'wavelet-LHH_gldm_SmallDependenceEmphasis', 'wavelet-HLH_gldm_DependenceEntropy', 'wavelet-HLH_gldm_DependenceNonUniformityNormalized', 'wavelet-HLH_gldm_DependenceVariance', 'wavelet-HLH_gldm_SmallDependenceEmphasis', 'wavelet-HHH_gldm_DependenceNonUniformityNormalized', 'wavelet-HHH_gldm_SmallDependenceEmphasis', 'wavelet-HHH_gldm_SmallDependenceHighGrayLevelEmphasis', 'wavelet-HHH_gldm_SmallDependenceLowGrayLevelEmphasis', 'wavelet-HHH_glrlm_RunLengthNonUniformityNormalized', 'wavelet-HHH_glrlm_ShortRunEmphasis', 'wavelet-LLL_firstorder_Energy', 'wavelet

In [6]:
radi_train.head()

,original_shape_SurfaceVolumeRatio,original_firstorder_Energy,original_firstorder_TotalEnergy,original_gldm_SmallDependenceEmphasis,original_glszm_LargeAreaEmphasis,original_glszm_ZoneVariance,wavelet-LHL_ngtdm_Coarseness,wavelet-LHH_gldm_DependenceEntropy,wavelet-LHH_gldm_DependenceNonUniformity,wavelet-LHH_gldm_DependenceNonUniformityNormalized,...,lbp-3D-m2_gldm_SmallDependenceLowGrayLevelEmphasis,lbp-3D-m2_glrlm_LongRunEmphasis,lbp-3D-m2_glrlm_LongRunHighGrayLevelEmphasis,lbp-3D-m2_glrlm_LongRunLowGrayLevelEmphasis,lbp-3D-m2_glrlm_RunLengthNonUniformityNormalized,lbp-3D-m2_glrlm_RunPercentage,lbp-3D-m2_glrlm_ShortRunEmphasis,lbp-3D-m2_glrlm_ShortRunHighGrayLevelEmphasis,lbp-3D-m2_glrlm_ShortRunLowGrayLevelEmphasis,Transition
121,0.646722,0.332975,0.332975,0.743187,0.165650,0.167302,0.577820,0.911714,0.259788,0.135969,...,0.528112,0.183653,0.183653,0.183653,0.633024,0.674847,0.658862,0.658862,0.658862,AD-AD
198,0.646124,0.279352,0.279352,0.605184,0.155027,0.152532,0.572456,1.000000,0.180384,0.024628,...,0.640870,0.216355,0.216355,0.216355,0.627178,0.663384,0.710960,0.710960,0.710960,AD-AD
60,0.142776,0.710948,0.710948,0.226206,0.453043,0.461957,0.154716,0.231668,0.715294,0.761206,...,0.132450,0.902781,0.902781,0.902781,0.098020,0.161192,0.249003,0.249003,0.249003,CN-CN
183,0.189767,0.634511,0.634511,0.164879,0.743082,0.748570,0.135693,0.124846,0.681030,0.818409,...,0.062202,0.667708,0.667708,0.667708,0.144705,0.201226,0.055939,0.055939,0.055939,MCI-MCI
136,0.464281,0.390720,0.390720,0.475275,0.233301,0.237776,0.410224,0.815456,0.342529,0.214382,...,0.390560,0.455996,0.455996,0.455996,0.414524,0.473195,0.514465,0.514465,0.514465,CN-CN


In [7]:
radi_test.head()

,original_shape_SurfaceVolumeRatio,original_firstorder_Energy,original_firstorder_TotalEnergy,original_gldm_SmallDependenceEmphasis,original_glszm_LargeAreaEmphasis,original_glszm_ZoneVariance,wavelet-LHL_ngtdm_Coarseness,wavelet-LHH_gldm_DependenceEntropy,wavelet-LHH_gldm_DependenceNonUniformity,wavelet-LHH_gldm_DependenceNonUniformityNormalized,...,lbp-3D-m2_gldm_SmallDependenceHighGrayLevelEmphasis,lbp-3D-m2_gldm_SmallDependenceLowGrayLevelEmphasis,lbp-3D-m2_glrlm_LongRunEmphasis,lbp-3D-m2_glrlm_LongRunHighGrayLevelEmphasis,lbp-3D-m2_glrlm_LongRunLowGrayLevelEmphasis,lbp-3D-m2_glrlm_RunLengthNonUniformityNormalized,lbp-3D-m2_glrlm_RunPercentage,lbp-3D-m2_glrlm_ShortRunEmphasis,lbp-3D-m2_glrlm_ShortRunHighGrayLevelEmphasis,lbp-3D-m2_glrlm_ShortRunLowGrayLevelEmphasis
0,0.321866,0.537274,0.537274,0.469422,0.257811,0.263728,0.262575,0.372223,0.521425,0.626043,...,0.199885,0.199885,0.470842,0.470842,0.470842,0.254125,0.335090,0.337339,0.337339,0.337339
1,0.162183,0.757994,0.757994,0.244068,0.533536,0.540881,0.142972,0.437059,0.705252,0.561655,...,0.114168,0.114168,0.854966,0.854966,0.854966,0.142767,0.183487,0.180486,0.180486,0.180486
2,0.225830,0.698128,0.698128,0.391731,0.355897,0.362740,0.146268,0.332013,0.677530,0.671927,...,0.263641,0.263641,0.602285,0.602285,0.602285,0.200483,0.233885,0.397251,0.397251,0.397251
3,0.200326,0.914195,0.914195,0.341228,0.479912,0.488448,0.061492,0.442873,0.756034,0.587332,...,0.255463,0.255463,0.670455,0.670455,0.670455,0.192451,0.205513,0.346410,0.346410,0.346410
4,0.227526,0.789830,0.789830,0.186788,0.514095,0.510353,0.179323,0.265433,0.620050,0.730427,...,0.137186,0.137186,0.625845,0.625845,0.625845,0.171591,0.237658,0.179175,0.179175,0.179175
